In [1]:
%load_ext autoreload
%autoreload 2

import sys, os
# if you're not launching jupyter from the repo root, fix the path:
repo_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)

In [2]:
from pydrake.all import StartMeshcat
meshcat = StartMeshcat()
print("Meshcat:", meshcat.web_url())

INFO:drake:Meshcat listening for connections at http://localhost:7000


Meshcat: http://localhost:7000


In [70]:
from rocky.build import build_robot_diagram
from rocky import utils
import os

urdf = os.path.join(repo_root, "rocky.urdf")   # or absolute path
bundle = build_robot_diagram(
    urdf_path=urdf,
    time_step=1e-3,
    gravity_vec=(0.,0.,0.),
    meshcat=meshcat,            # <- pass meshcat from notebook
)

bundle = build_robot_diagram(urdf, time_step=1e-3, gravity_vec=(0,0,0), meshcat=meshcat)
plant, plant_context, sim = bundle.plant, bundle.plant_context, bundle.simulator

utils.set_joint_positions(plant, plant_context, [0.0, 0.0])
utils.set_zero_torques(plant, plant_context)
utils.advance(sim, 2.0)